インポート

In [1]:
from pyknp import Juman
from pyknp import KNP
import pandas as pd

文，KNP用意

In [9]:
sentence = ""
knp = KNP()

In [10]:
file_data = pd.read_csv("all_data.csv", header=None)

In [11]:
esa_result = []
para = False

述語項構造&構文木

In [12]:
class evaluation_structure_automatic_construction():
    def __init__(self,sentence):
        self.setsu_n, self.setsu_b, self.setsu_f = 0, 0, 0 #節区切の番号
        self.setsu_del_b, self.setsu_del_f, self.setsu_del_n = 0, 0, 0 #クエリ削除の番号
        self.setsu_temp = ""
        self.result = knp.parse(sentence)
        self.setsu_flg = True
        self.setsu_list = list()
        self.setsu_del = list()
    
    def danwa_setsu_set(self,sentence):
        self.setsu_b = self.result.get_tag_span(self.setsu_n)[0] #節位置の先頭
        self.setsu_f = self.result.get_tag_span(tag.tag_id)[1] #節位置の末端
        # print("節",self.setsu_b,self.setsu_f)
        if self.setsu_f - self.setsu_b > 4:
            for i in range(self.setsu_b,self.setsu_f + 1): 
                for j in range(len(self.setsu_del)):
                    if(i == self.setsu_del[j]):
                        self.setsu_flg = False
                        break

                if self.setsu_flg == True:
                    self.setsu_temp += sentence[i] #節に格納
                self.setsu_flg = True

            self.setsu_list.insert(0, self.setsu_temp) #節リストに格納
            # print("\n",self.setsu_temp,"\n")
            self.setsu_temp = ""
        
        
    

In [13]:
# ファイルの内容を一行ずつ読み込む
for j in range(24):
    for i in range(len(file_data)):
        sentence = file_data[j][i]
        # print(sentence)

        setsu = evaluation_structure_automatic_construction(sentence)
        temp = 0
        for tag in setsu.result.tag_list(): # 各基本句へのアクセス
            para = False
            # print("\tID:%d, 見出し:%s, 係り受けタイプ:%s, 親基本句ID:%d, 素性:%s" \
            #         % (tag.tag_id, "".join(mrph.midasi for mrph in tag.mrph_list()), tag.dpndtype, tag.parent_id, tag.fstring)) 
            
            if '節-区切' in tag.fstring or '節-機能-原因・理由' in tag.fstring or '節-機能疑-原因・理由' in tag.fstring or 'ID:〜によって' in tag.fstring: #第1段階
                if not '連体修飾' in tag.fstring and not '主題表現' in tag.fstring: 
                    # print(tag.tag_id)
                    temp = tag.tag_id
                    setsu.danwa_setsu_set(sentence)
                    setsu.setsu_n = tag.tag_id + 1

            elif ('<解析連格:外の関係>' in tag.fstring or '<外の関係>' in tag.fstring) and '<連用要素>' in tag.fstring: #第2段階
                # print("Temp" ,tag.tag_id - temp)
                if tag.tag_id - temp > 2: 
                    setsu.danwa_setsu_set(sentence)
                    setsu.setsu_n = tag.tag_id + 1

            elif '並列タイプ:AND' in tag.fstring and not '<連用要素>' in tag.fstring:
                # print("Temp" ,tag.tag_id - temp)
                if tag.tag_id - temp > 2:
                    para = True
                    setsu.danwa_setsu_set(sentence)
                    setsu.setsu_n = tag.tag_id + 1

        esa_result.append([sentence])
        esa_result.append([" ― ".join(setsu.setsu_list)])

In [14]:
esa_result = pd.DataFrame(esa_result)
esa_result

,0
0,スピーカーの左右の音の出し方を工夫するため良いスピーカーを使うと、スピーカーから聞こえる音に...
1,スピーカーから聞こえる音にライブ感を感じる。 ― 良いスピーカーを使うと、 ― 工夫するため...
2,画面構成を固定する事が必要。そのために臨場感のある音の広がりが必要なのはステージを主体とした...
3,その映像はよりライブ感を感じる。 ― ステージを主体とした観客席の視点からだったから。 ― ...
4,なるべく一人になれる時間を作る。そのために静かな場所で聴く。その結果、音質のよさがライブ感を...
...,...
475,ライブ感を感じる。 ― ライブのグッツ購入ができることに ― グッツの販売ｗｅｂサイトでライ...
476,会場全体を包むような楽曲に対する共感が得られることにより、楽曲に身を委ねた一体感が感じられ、...
477,ライブ感を感じる。 ― パフォーマーによるオーディエンスへの呼び掛けがあることで ― 楽曲に...
478,パフォーマーの個性や楽曲イメージに合う舞台設定によってライトやスモークを使っての魅惑的な演出...


In [15]:
esa_result.to_csv("result_02021430.csv")

In [26]:
sentence = "実際の観客を動員する為のチケット販売や、昨今のご時勢に配慮した健康面での安全対策を行った上で、そのライブ会場を埋め尽くす、もしくは7～8割は埋めるくらいの大勢の観客が必要。それらの大勢の観客の、演奏の終盤や演奏後における興奮気味の歓声、応援を含んだライブ映像からはライブ感を感じる。"
print(sentence)

setsu = evaluation_structure_automatic_construction(sentence)
para = False
temp = 0

for tag in setsu.result.tag_list(): # 各基本句へのアクセス
    para = False
    print("\tID:%d, 見出し:%s, 係り受けタイプ:%s, 親基本句ID:%d, 素性:%s" \
            % (tag.tag_id, "".join(mrph.midasi for mrph in tag.mrph_list()), tag.dpndtype, tag.parent_id, tag.fstring)) 

    if '節-区切' in tag.fstring or '節-機能-原因・理由' in tag.fstring or '節-機能疑-原因・理由' in tag.fstring or 'ID:〜によって' in tag.fstring: #第1段階
        if not '連体修飾' in tag.fstring and not '主題表現' in tag.fstring: 
            print(tag.tag_id)
            temp = tag.tag_id
            setsu.danwa_setsu_set(sentence)
            setsu.setsu_n = tag.tag_id + 1

    elif ('<解析連格:外の関係>' in tag.fstring or '<外の関係>' in tag.fstring) and '<連用要素>' in tag.fstring: #第2段階
        print("Temp" ,tag.tag_id - temp)
        if tag.tag_id - temp > 2: 
            setsu.danwa_setsu_set(sentence)
            setsu.setsu_n = tag.tag_id + 1

    elif '並列タイプ:AND' in tag.fstring and not '<連用要素>' in tag.fstring: #第3段階
        print("Temp" ,tag.tag_id - temp)
        if tag.tag_id - temp > 2:
            para = True
            setsu.danwa_setsu_set(sentence)
            setsu.setsu_n = tag.tag_id + 1

esa_result.append([sentence])
esa_result.append([" ― ".join(setsu.setsu_list)])


print([" ― ".join(setsu.setsu_list)])
print("\n")

実際の観客を動員する為のチケット販売や、昨今のご時勢に配慮した健康面での安全対策を行った上で、そのライブ会場を埋め尽くす、もしくは7～8割は埋めるくらいの大勢の観客が必要。それらの大勢の観客の、演奏の終盤や演奏後における興奮気味の歓声、応援を含んだライブ映像からはライブ感を感じる。
	ID:0, 見出し:実際の, 係り受けタイプ:D, 親基本句ID:1, 素性:<BGH:実際/じっさい><文頭><助詞><連体修飾><体言><係:ノ格><区切:0-4><名詞項候補><先行詞候補><係チ:非用言格解析||用言&&文節内:Ｔ解析格-ヲ><正規化代表表記:実際/じっさい><主辞代表表記:実際/じっさい>
	ID:1, 見出し:観客を, 係り受けタイプ:D, 親基本句ID:2, 素性:<SM-主体><SM-人><BGH:観客/かんきゃく><ヲ><助詞><体言><係:ヲ格><区切:0-0><格要素><連用要素><名詞項候補><先行詞候補><正規化代表表記:観客/かんきゃく><主辞代表表記:観客/かんきゃく><解析格:ヲ>
	ID:2, 見出し:動員する, 係り受けタイプ:D, 親基本句ID:3, 素性:<BGH:動員/どういん+する/する><サ変動詞><連体修飾><用言:動><係:連格><レベル:B><区切:0-5><ID:〜（ため）><連体節><動態述語><サ変><正規化代表表記:動員/どういん><主辞代表表記:動員/どういん><用言代表表記:動員/どういん><時制:非過去><格関係1:ヲ:観客><格関係3:外の関係:為><格解析結果:動員/どういん:動2:ガ/U/-/-/-/-;ヲ/C/観客/1/0/251;ニ/U/-/-/-/-;デ/U/-/-/-/-;時間/U/-/-/-/-;外の関係/N/為/3/0/251><標準用言代表表記:動員/どういん>
	ID:3, 見出し:為の, 係り受けタイプ:D, 親基本句ID:7, 素性:<形副名詞><ため-せい><外の関係><助詞><連体修飾><体言><一文字漢字><係:連体><レベル:B><区切:3-5><機能的基本句><名詞項候補><省略解析なし><正規化代表表記:為/ため><主辞代表表記:為/ため><クエリ削除語><解析連格:外の関係>
	ID:4, 見出し:チケット, 係り受けタイプ:D, 親基本句ID:5, 素性:<BGH